In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# Configuration
import os
import sys
rootpath = os.path.dirname(os.getcwd())
sys.path.append(rootpath)

from news import NewsIO, NewsFunc, NewsPath
newsio = NewsIO()
newspath = NewsPath()
newsfunc = NewsFunc()

import pandas as pd
from tqdm import tqdm
from copy import deepcopy
from collections import defaultdict

In [2]:
## Parameters
TOPN = 1000

/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
## Filenames
fname_target_word_list = 'target_word_list.json'

fname_text_feature_first = 'text_feature_first.json'
fname_text_feature_second = 'text_feature_second.json'
fname_text_feature_topic_first = 'text_feature_topic_first.json'
fname_text_feature_topic_second = 'text_feature_topic_second.json'

fname_numeric_feature_first = 'numeric_feature_first.json'
fname_numeric_feature_second = 'numeric_feature_second.json'

fname_data = f'data_w-{TOPN}.pk'
fname_data_norm = f'data_w-{TOPN}_norm.pk'

/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
target_word_list = [w for w, v in newsio.load_json(fname_object=fname_target_word_list, _type='data')][:TOPN]

text_feature_first = newsio.load_json(fname_object=fname_text_feature_first, _type='data')
text_feature_second = newsio.load_json(fname_object=fname_text_feature_second, _type='data')
text_feature_topic_first = newsio.load_json(fname_object=fname_text_feature_topic_first, _type='data')
text_feature_topic_second = newsio.load_json(fname_object=fname_text_feature_topic_second, _type='data')

/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  | fdir : /data/blank54/workspace/project/news/data
  | fname: target_word_list.json
  | fdir : /data/blank54/workspace/project/news/data
  | fname: text_feature_first.json
  | fdir : /data/blank54/workspace/project/news/data
  | fname: text_feature_second.json
  | fdir : /data/blank54/workspace/project/news/data
  | fname: text_feature_topic_first.json
  | fdir : /data/blank54/workspace/project/news/data
  | fname: text_feature_topic_second.json


In [7]:
numeric_feature_first = newsio.load_json(fname_object=fname_numeric_feature_first, _type='data')
numeric_feature_second = newsio.load_json(fname_object=fname_numeric_feature_second, _type='data')

/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  | fdir : /data/blank54/workspace/project/news/data
  | fname: numeric_feature_first.json
  | fdir : /data/blank54/workspace/project/news/data
  | fname: numeric_feature_second.json


In [8]:
yearmonth_list = list(text_feature_first.keys())
topic_list = list(text_feature_topic_first['200501']['count'].keys())
numeric_variable_list = list(numeric_feature_first['200501']['raw'].keys())

data = defaultdict(list)
for yearmonth in tqdm(yearmonth_list[1:]):
    data['yearmonth'].append(yearmonth)
    
    ## Words
    for word in target_word_list:
        try:
            data[f'word_count_{word}'].append(text_feature_first[yearmonth]['word_count'][word])
        except KeyError:
            data[f'word_count_{word}'].append(0)
            
        try:
            data[f'word_count_portion_{word}'].append(text_feature_first[yearmonth]['word_count_portion'][word])
        except KeyError:
            data[f'word_count_portion_{word}'].append(0)
            
        try:
            data[f'word_count_diff_{word}'].append(text_feature_second[yearmonth]['word_count_diff'][word])
        except KeyError:
            data[f'word_count_diff_{word}'].append(0)
            
        try:
            data[f'word_count_ratio_{word}'].append(text_feature_second[yearmonth]['word_count_ratio'][word])
        except KeyError:
            data[f'word_count_ratio_{word}'].append(0)
            
        try:
            data[f'word_count_portion_diff_{word}'].append(text_feature_second[yearmonth]['word_count_portion_diff'][word])
        except KeyError:
            data[f'word_count_portion_diff_{word}'].append(0)
            
        try:
            data[f'word_count_portion_ratio_{word}'].append(text_feature_second[yearmonth]['word_count_portion_ratio'][word])
        except KeyError:
            data[f'word_count_portion_ratio_{word}'].append(0)
            
    ## Documents
    data['doc_count'].append(text_feature_first[yearmonth]['doc_count'])
    data['doc_count_diff'].append(text_feature_second[yearmonth]['doc_count_diff'])
    data['doc_count_ratio'].append(text_feature_second[yearmonth]['doc_count_ratio'])
        
    ## Topics
    for topic_id in topic_list:
        try:
            data[f'topic_count_{topic_id}'].append(text_feature_topic_first[yearmonth]['count'][topic_id])
        except KeyError:
            data[f'topic_count_{topic_id}'].append(0)
            
        try:
            data[f'topic_count_portion_{topic_id}'].append(text_feature_topic_first[yearmonth]['portion'][topic_id])
        except KeyError:
            data[f'topic_count_portion_{topic_id}'].append(0)
        
        data[f'topic_count_diff_{topic_id}'].append(text_feature_topic_second[yearmonth]['diff'][topic_id])
        data[f'topic_count_ratio_{topic_id}'].append(text_feature_topic_second[yearmonth]['ratio'][topic_id])
        
    ## Numerics
    for var in numeric_variable_list:
        data[f'numeric_raw_{var}'].append(numeric_feature_first[yearmonth]['raw'][var])
        data[f'numeric_diff_{var}'].append(numeric_feature_second[yearmonth]['diff'][var])
        data[f'numeric_ratio_{var}'].append(numeric_feature_second[yearmonth]['ratio'][var])
        
df = pd.DataFrame(data)

/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 179/179 [00:00<00:00, 179.32it/s]


In [9]:
newsio.save(_object=df, _type='data', fname_object=fname_data)
df = newsio.load(fname_object=fname_data, _type='data')

df

  | fdir : /data/blank54/workspace/project/news/data
  | fname: data_w-1000.pk
  | fdir : /data/blank54/workspace/project/news/data
  | fname: data_w-1000.pk


/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,yearmonth,word_count_사업,word_count_portion_사업,word_count_diff_사업,word_count_ratio_사업,word_count_portion_diff_사업,word_count_portion_ratio_사업,word_count_가구,word_count_portion_가구,word_count_diff_가구,...,numeric_ratio_pc_공공기관,numeric_raw_pc_민간기관,numeric_diff_pc_민간기관,numeric_ratio_pc_민간기관,numeric_raw_pc_민자,numeric_diff_pc_민자,numeric_ratio_pc_민자,numeric_raw_pc_국내외국기관,numeric_diff_pc_국내외국기관,numeric_ratio_pc_국내외국기관
0,200502,2674,0.005039,-1007,-1007,-0.000908,-0.000908,2340,0.004410,-623,...,0.742918,0.000000,-0.038163,0.000000,0.000000,-0.077483,0.000000,0.063651,-0.039552,0.616755
1,200503,4572,0.004996,1898,1898,-0.000043,-0.000043,4478,0.004894,2138,...,3.089941,0.139986,0.139986,0.000014,0.150398,0.150398,0.000015,0.086612,0.022960,1.360721
2,200504,5727,0.006668,1155,1155,0.001672,0.001672,3939,0.004586,-539,...,1.142790,0.159080,0.019094,1.136397,0.145034,-0.005364,0.964333,0.087889,0.001277,1.014743
3,200505,6021,0.006767,294,294,0.000099,0.000099,3740,0.004203,-199,...,1.051709,0.176901,0.017822,1.112029,0.140669,-0.004364,0.969907,0.102952,0.015063,1.171386
4,200506,4697,0.005826,-1324,-1324,-0.000940,-0.000940,3290,0.004081,-450,...,1.340406,0.208920,0.032019,1.180999,0.204734,0.064065,1.455428,0.114867,0.011915,1.115737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,201908,16314,0.006360,-4112,-4112,-0.000575,-0.000575,6816,0.002657,-346,...,1.047895,0.759346,-0.002612,0.996572,0.246813,-0.088698,0.735633,0.084863,-0.026474,0.762218
175,201909,17315,0.007092,1001,1001,0.000731,0.000731,7393,0.003028,577,...,1.078912,0.704842,-0.054503,0.928223,0.455769,0.208956,1.846617,0.118511,0.033648,1.396492
176,201910,21142,0.006685,3827,3827,-0.000407,-0.000407,7451,0.002356,58,...,1.117924,0.777915,0.073073,1.103673,0.326850,-0.128918,0.717141,0.052647,-0.065864,0.444239
177,201911,24422,0.007489,3280,3280,0.000804,0.000804,7336,0.002250,-115,...,1.168406,0.777407,-0.000508,0.999347,0.309143,-0.017707,0.945824,0.117038,0.064392,2.223080


In [10]:
demographic_info_df = newsfunc.explore_demographic_info(df=df, except_list=['yearmonth'])
variable_list_meanstd, variable_list_minmax = newsfunc.partition_variable_list(demographic_info_df)

/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Error: every element is zero: numeric_raw_rcc_강원국내외국기관
Error: every element is zero: numeric_ratio_rcc_강원국내외국기관


In [11]:
print('## Mean-standard normalization')
print('  | Variables: {:,}'.format(len(variable_list_meanstd)))
df_meanstd = deepcopy(df[variable_list_meanstd])
df_meanstd_norm = newsfunc.normalize_meanstd(df_meanstd)
print('  | data shape: {}'.format(df_meanstd_norm.shape))
df_meanstd_norm = deepcopy(pd.concat([df_meanstd_norm, df['yearmonth']], axis=1))

print('## Min-max normalization')
print('  | Variables: {:,}'.format(len(variable_list_minmax)))
df_minmax = deepcopy(df[variable_list_minmax])
df_minmax_norm = newsfunc.normalize_minmax(df_minmax)
print('  | data shape: {}'.format(df_minmax_norm.shape))
df_minmax_norm = deepcopy(pd.concat([df_minmax_norm, df['yearmonth']], axis=1))

print('## Merge normalized data')
df_norm = deepcopy(pd.merge(df_meanstd_norm, df_minmax_norm))
print('  | data shape: {}'.format(df_norm.shape))

/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Mean-standard normalization
  | Variables: 4,316
  | data shape: (179, 4316)
## Min-max normalization
  | Variables: 2,561
  | data shape: (179, 2561)
## Merge normalized data
  | data shape: (179, 6878)


In [12]:
newsio.save(_object=df_norm, _type='data', fname_object=fname_data_norm)
df_norm = newsio.load(fname_object=fname_data_norm, _type='data')

df_norm

  | fdir : /data/blank54/workspace/project/news/data
  | fname: data_w-1000_norm.pk
  | fdir : /data/blank54/workspace/project/news/data
  | fname: data_w-1000_norm.pk


/data/blank54/anaconda3/envs/news/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,word_count_diff_사업,word_count_ratio_사업,word_count_portion_diff_사업,word_count_portion_ratio_사업,word_count_diff_가구,word_count_ratio_가구,word_count_portion_diff_가구,word_count_portion_ratio_가구,word_count_diff_분양,word_count_ratio_분양,...,numeric_raw_pc_기성총액,numeric_ratio_pc_기성총액,numeric_raw_pc_공공기관,numeric_ratio_pc_공공기관,numeric_raw_pc_민간기관,numeric_ratio_pc_민간기관,numeric_raw_pc_민자,numeric_ratio_pc_민자,numeric_raw_pc_국내외국기관,numeric_ratio_pc_국내외국기관
0,-0.451448,-0.451448,-1.209901,-1.209901,-0.564321,-0.564321,-0.711308,-0.711308,-0.129465,-0.129465,...,0.000000,0.000000,0.056336,0.240431,0.000000,0.000000e+00,0.000000,0.000000,0.063651,0.022597
1,0.704111,0.704111,-0.070625,-0.070625,1.851721,1.851721,0.980236,0.980236,1.299048,1.299048,...,0.150700,0.000003,0.174075,1.000000,0.139986,3.831932e-07,0.150398,0.000005,0.086612,0.049855
2,0.408558,0.408558,2.186128,2.186128,-0.490816,-0.490816,-0.573677,-0.573677,0.074488,0.074488,...,0.172809,0.210141,0.198932,0.369842,0.159080,3.110737e-02,0.145034,0.309130,0.087889,0.037179
3,0.066068,0.066068,0.115609,0.115609,-0.193296,-0.193296,-0.721976,-0.721976,-0.527301,-0.527301,...,0.189258,0.200699,0.209218,0.340365,0.176901,3.044035e-02,0.140669,0.310917,0.102952,0.042918
4,-0.577545,-0.577545,-1.252002,-1.252002,-0.412936,-0.412936,-0.209593,-0.209593,-0.513872,-0.513872,...,0.240466,0.232838,0.280437,0.433797,0.208920,3.232830e-02,0.204734,0.466557,0.114867,0.040879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,-1.686564,-1.686564,-0.771384,-0.771384,-0.321929,-0.321929,0.473005,0.473005,-0.800078,-0.800078,...,0.626401,0.182229,0.227401,0.339131,0.759346,2.727985e-02,0.246813,0.235817,0.084863,0.027927
175,0.347300,0.347300,0.948297,0.948297,0.485752,0.485752,0.757650,0.757650,-0.927654,-0.927654,...,0.604479,0.176842,0.245346,0.349169,0.704842,2.540890e-02,0.455769,0.591958,0.118511,0.051165
176,1.471434,1.471434,-0.549913,-0.549913,0.031595,0.031595,-1.289453,-1.289453,-0.231862,-0.231862,...,0.659870,0.200047,0.274278,0.361795,0.777915,3.021161e-02,0.326850,0.229889,0.052647,0.016276
177,1.253847,1.253847,1.044619,1.044619,-0.119790,-0.119790,-0.178536,-0.178536,0.034201,0.034201,...,0.674979,0.187451,0.320468,0.378132,0.777407,2.735582e-02,0.309143,0.303197,0.117038,0.081450
